# Lien entre performance énergétique des logements (DPE) et niveau de vie des ménages
## Une analyse à l'échelle des communes d'Ile-de-France

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1) Récupération des données:

### Données de l'ADEME (DPE)

In [45]:
# Récupération des données de l'ADEME portant sur la performance énergétique des logements résidentiels situés dans
# les 8 départements d'Ile-de-France:
Paris=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-75/data-files/dpe-75.csv")
Seine_et_Marne=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-77/data-files/dpe-77.csv")
Yvelines=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-78/data-files/dpe-78.csv")
Essonne=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-91/data-files/dpe-91.csv")
Hauts_de_Seine=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-92/data-files/dpe-92.csv")
Seine_saint_Denis=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-93/data-files/dpe-93.csv")
Val_de_Marne=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-94/data-files/dpe-94.csv")
Val_Oise=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-95/data-files/dpe-95.csv")

/tmp/ipykernel_501/3939422412.py:3: DtypeWarning: Columns (8,35,36,37,43,76,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  Paris=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-75/data-files/dpe-75.csv")
/tmp/ipykernel_501/3939422412.py:4: DtypeWarning: Columns (8,27,35,36,37,39,43,76,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  Seine_et_Marne=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-77/data-files/dpe-77.csv")
/tmp/ipykernel_501/3939422412.py:5: DtypeWarning: Columns (35,36,37,43,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  Yvelines=pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/dpe-78/data-files/dpe-78.csv")
/tmp/ipykernel_501/3939422412.py:6: DtypeWarning: Columns (35,36,37,43,76,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  Essonne=pd.read_csv("https://koumoul.com/data-fair/ap

In [46]:
DPE_IDF=pd.concat([Paris,Seine_et_Marne,Yvelines,Essonne,Hauts_de_Seine,Seine_saint_Denis,Val_de_Marne,Val_Oise])

In [47]:
# Liste de toutes les variables dans le fichier:
print(DPE_IDF.columns.tolist())

['id', 'numero_dpe', 'usr_diagnostiqueur_id', 'usr_logiciel_id', 'tr001_modele_dpe_id', 'nom_methode_dpe', 'version_methode_dpe', 'nom_methode_etude_thermique', 'version_methode_etude_thermique', 'date_visite_diagnostiqueur', 'date_etablissement_dpe', 'date_arrete_tarifs_energies', 'commentaires_ameliorations_recommandations', 'explication_personnalisee', 'consommation_energie', 'classe_consommation_energie', 'estimation_ges', 'classe_estimation_ges', 'tr002_type_batiment_id', 'secteur_activite', 'tr012_categorie_erp_id', 'tr013_type_erp_id', 'annee_construction', 'surface_habitable', 'surface_thermique_lot', 'tv016_departement_id', 'commune', 'arrondissement', 'type_voie', 'nom_rue', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'code_postal', 'code_insee_commune', 'code_insee_commune_actualise', 'numero_lot', 'quote_part', 'nom_centre_commercial', 'surface_commerciale_contractuelle', 'portee_dpe_batiment', 'partie_batiment', 'shon', 'surface_utile', 'surface_thermique_parti

In [48]:
# On restreint le dadatframe DPE_IDF aux variables pertinentes pour notre analyse:
DPE_IDF=DPE_IDF.loc[:,["id","numero_dpe","date_visite_diagnostiqueur","consommation_energie","classe_consommation_energie","estimation_ges","classe_estimation_ges","annee_construction","surface_habitable","surface_thermique_lot","commune","code_insee_commune","longitude","latitude","tr002_type_batiment_description","tv016_departement_departement"]]
DPE_IDF.head()

,id,numero_dpe,date_visite_diagnostiqueur,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,annee_construction,surface_habitable,surface_thermique_lot,commune,code_insee_commune,longitude,latitude,tr002_type_batiment_description,tv016_departement_departement
0,176,1375V2000001D,2013-04-10,249.00,E,15.00,C,1947,54.00,54.00,PARIS,75013.0,2.416435,48.930427,Logement,75 - Paris
1,219,1375V2000003F,2013-04-09,692.00,G,22.00,D,1947,23.86,23.86,PARIS,75019.0,6.055365,46.332839,Logement,75 - Paris
2,220,1375V2000004G,2013-04-10,324.00,E,75.00,F,1947,54.89,54.89,PARIS,75012.0,NaN,NaN,Logement,75 - Paris
3,248,1375V2000008K,2013-04-04,377.68,F,12.29,C,1850,38.90,38.90,PARIS,75112.0,3.244265,45.755699,Logement,75 - Paris
4,250,1375V2000009L,2013-04-11,309.00,E,10.00,B,1,80.00,80.00,PARIS,75012.0,2.391975,48.839267,Logement,75 - Paris


### Données de l'INSEE (Fichier FILOSOFI_COM des revenus localisés sociaux et fiscaux en 2016 à l'échelle communale) 

In [19]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 7.2 MB/s eta 0:00:00
  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-yd2krl7u
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-yd2krl7u
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit ebebbb6b95868b98c0ac2fede3080797215f47b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee
import pynsee.download

In [23]:
np.random.seed(123)

In [24]:
df_city = pynsee.download.download_file("FILOSOFI_COM_2016")

Extracting: 100%|██████████| 10.6M/10.6M [00:00<00:00, 138MB/s]


In [25]:
df_city.head()

,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,PIMP16,TP6016,TP60AGE116,TP60AGE216,TP60AGE316,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
0,01001,L'Abergement-Clémenciat,313,795.5,22679,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,101,248,24382.083333333336,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01004,Ambérieu-en-Bugey,6363,14228,19721,49,17,19,22,22,...,27,8.8,6.9,2.8,2.1,2,-15.7,10457.083333333334,33880.555555555555,3.239962279688143
3,01005,Ambérieux-en-Dombes,633,1662.5,23378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01006,Ambléon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2) Premiers retraitements sur les données

### Données de l'ADEME

In [49]:
print(DPE_IDF.columns.tolist())

['id', 'numero_dpe', 'date_visite_diagnostiqueur', 'consommation_energie', 'classe_consommation_energie', 'estimation_ges', 'classe_estimation_ges', 'annee_construction', 'surface_habitable', 'surface_thermique_lot', 'commune', 'code_insee_commune', 'longitude', 'latitude', 'tr002_type_batiment_description', 'tv016_departement_departement']


In [50]:
print(DPE_IDF.dtypes)

id                                   int64
numero_dpe                          object
date_visite_diagnostiqueur          object
consommation_energie               float64
classe_consommation_energie         object
estimation_ges                     float64
classe_estimation_ges               object
annee_construction                   int64
surface_habitable                  float64
surface_thermique_lot              float64
commune                             object
code_insee_commune                  object
longitude                          float64
latitude                           float64
tr002_type_batiment_description     object
tv016_departement_departement       object
dtype: object


In [51]:
print(DPE_IDF.shape)

(1587477, 16)


In [52]:
# Création d'une variable année:
DPE_IDF["annee"]=DPE_IDF.date_visite_diagnostiqueur.str[:4]

In [53]:
DPE_IDF.groupby('annee')["code_insee_commune"].count()

annee
0217         1
1013         1
1015         1
1016         3
1753         1
1899        12
1900         1
1932         1
1970         1
1992         1
2000       302
2001         3
2002         2
2003         3
2004         7
2005         2
2006         1
2007         3
2008         7
2009        15
2010        12
2011        55
2012       210
2013     91829
2014    185834
2015    196484
2016    209376
2017    203677
2018    216520
2019    236762
2020    218348
2021     25390
2022         1
2023         1
2024         1
2026         1
2030         1
2041         2
2103        11
2104         5
2105         4
2106         2
2107         4
2108         1
2109         2
2116         2
2130         2
2140         1
2165         1
2213         1
3013         3
3014         1
9999         1
Name: code_insee_commune, dtype: int64

### Il en ressort que le fichier des DPE IDF semble plutôt exploitable sur les années 2014-2020.

In [54]:
# Premier filtrage sur les lignes: on ne conserve que les DPE réalisés en 2016
DPE_IDF_16 = DPE_IDF[DPE_IDF["annee"]=="2016"].copy()

In [55]:
print(DPE_IDF_16.shape)

(209583, 17)


In [56]:
DPE_IDF_16.head()

,id,numero_dpe,date_visite_diagnostiqueur,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,annee_construction,surface_habitable,surface_thermique_lot,commune,code_insee_commune,longitude,latitude,tr002_type_batiment_description,tv016_departement_departement,annee
51601,1344868,1475L2009870A,2016-09-17,135.0,C,31.00,D,1947,57.00,57.00,PARIS,75056.0,2.391569,48.849544,Logement,75 - Paris,2016
67427,1859286,1575V2002830R,2016-02-26,0.0,N,0.00,N,1947,26.10,26.10,PARIS,75056.0,2.390157,48.838045,Logement,75 - Paris,2016
102239,2913865,1575V2019251W,2016-12-29,340.5,F,11.08,C,1947,51.66,51.66,PARIS,75117,2.325752,48.890702,Logement,75 - Paris,2016
102494,2917324,1675V2000004J,2016-01-01,321.0,E,75.00,F,1947,87.88,87.88,PARIS - 11EME,75056,2.378457,48.862742,Logement,75 - Paris,2016
102495,2917330,1675V2000005K,2016-01-01,188.0,D,43.00,E,1947,29.02,29.02,PARIS - 11EME,75056,2.377194,48.866060,Logement,75 - Paris,2016


### On retient dans un premier temps l'année 2016 pour l'analyse des DPE, car on souhaite in fine rapprocher ces données du fichier Filosofi (au niveau communal) de l'Insee dont la dernière version disponible correspond au millésime 2016.

In [44]:
print(Paris_DPE_16[['code_insee_commune', 'commune']].nunique())

code_insee_commune    757
commune               881
dtype: int64
